# 회원 보유종목 데이터

### 고은세님 요청사항: 운용팀에서 예상하는 보유 종목 이외의 다른 종목을 보유한 고객정보
### columns: 회원 아이디, 전략, 종목

### 02-26 추가사항
##### 1. 띄어씌기 여부와 상관없이 필터 가능
##### 2. 회원이름 추가
##### 3. 스프레드 시트로부터 보유종목 xlsx 파일 생성
##### 4. 생성된 파일의 종목명 오류 체크
##### 5. STRPURSTARTTIME 15:30 이상인 사람 제외
##### 6. A,C,E,S,U,N,G,T,SVIP6,VVIP1 전략 한에서 검색
##### 7. 전략명과 종목명 묶어서

In [9]:
import cjw_maria
import pandas as pd
import datetime

host = "15.165.29.213"
user = "lt_user"
password = "De4IjOY32e7o" 
db = "leaderstrading"
maria = cjw_maria.MariaDB(host, user, password, db)

sql = 'SELECT NAME, ID, ClientID, CompleteTime, StrategyName, TradingType, Status \
    FROM trading_history \
    ORDER BY ClientID, NAME, CompleteTime ASC;'
ordered_df = maria.showData(sql)

after_processed_df = ordered_df.drop_duplicates(subset=['NAME', 'ClientID'], keep='last')

after_processed_df.value_counts('TradingType')

have_df = after_processed_df[after_processed_df['TradingType']=='매수'].reset_index(drop=True)
have_df.drop_duplicates(subset=['ID', 'NAME'])[['ID', 'NAME']].to_csv('history_name.csv', index=False, encoding='utf-8-sig')

df = have_df[['ClientID', 'NAME', 'StrategyName']]

df['NAME'] = pd.DataFrame(df['NAME']).applymap(str.upper)['NAME']
tmp = list(df['NAME'])
df['NAME'] = [t.replace(' ','') for t in tmp]

boyoo = pd.read_excel('보유종목.xlsx', sheet_name='보유종목')
boyoo.columns = boyoo.iloc[3]
boyoo = boyoo.iloc[5:]
boyoo.columns = [c.replace(' ','') if type(c)==str else c for c in list(boyoo.columns)]

tmp = ['VVIP1', 'SVIP6', 'G', 'E', 'A', 'C', 'S', 'N', 'T', 'S\'']

cols = []
for i in range(len(boyoo.columns)):
    if boyoo.columns[i] in tmp:
        cols.append(boyoo.columns[i])
        cols.append(boyoo.columns[i+1])

boyoo = boyoo[cols]
woonyong = []
print(list(boyoo.columns))

for i in range(0, len(list(boyoo.columns)), 2):
    data = list(boyoo[boyoo.columns[i:i+2]].dropna()[boyoo.columns[i]])
    for d in data:
        if len(boyoo.columns[i])<=2:
            woonyong.append(['Master.'+boyoo.columns[i], d])
        else:
            woonyong.append([boyoo.columns[i], d])

woonyong = pd.DataFrame(woonyong, columns=['전략명', '종목명'])

woonyong['종목명'] = pd.DataFrame(woonyong['종목명']).applymap(str.upper)['종목명']
tmp = list(woonyong['종목명'])
woonyong['종목명'] = [t.replace(' ','') for t in tmp]
woonyong = woonyong.reset_index(drop=True)

u_start = woonyong[woonyong['종목명']=='U'].index[0]


tmp = woonyong.iloc[u_start:].reset_index(drop=True)

for i in range(1, tmp.shape[0]):
    if tmp['전략명'][i] != tmp['전략명'][i-1]:
        break

woonyong = woonyong.drop(u_start)
woonyong = woonyong.drop(u_start+1)
woonyong.iloc[u_start:u_start+i-2]['전략명'] = ['Master.U'] * (i-2)
woonyong = woonyong.reset_index(drop=True)

nowtime = '보유종목'+str(datetime.datetime.now()).replace(' ', '_').replace(':', '.') + '.csv'
woonyong.to_csv(nowtime, encoding='utf-8-sig', index=False)

woonyong['종목명'] = pd.DataFrame(woonyong['종목명']).applymap(str.upper)['종목명']
tmp = list(woonyong['종목명'])
woonyong['종목명'] = [t.replace(' ','') for t in tmp]
woonyong = woonyong.reset_index(drop=True)

error_name = []
for i in range(woonyong.shape[0]):
    if woonyong['종목명'][i] not in list(df.drop_duplicates('NAME')['NAME']):
        error_name.append(woonyong['종목명'][i])

pd.DataFrame(error_name, columns=['종목명']).to_csv('error_name.csv', index=False, encoding='utf-8-sig')

stradegies = ['Master.A','Master.C','Master.E',
              'Master.S','Master.U','Master.N',
              'Master.G','Master.T','SVIP6', 'VVIP1']

indexNames = []
for i in range(woonyong['전략명'].shape[0]):
    if woonyong['전략명'].iloc[i] in stradegies:
        indexNames.append(i)


woonyong = woonyong.iloc[indexNames].reset_index(drop=True)

res = []

df['new'] = df['StrategyName'].astype(str) + df['NAME']

woonyong = woonyong.replace('Master.S\'', 'Master.S')
woonyong['new'] = woonyong['전략명'].astype(str) + woonyong['종목명']

for n in list(df.drop_duplicates('new')['new']):
    if n not in list(woonyong['new']):
        res.append(df[df['new']==n][['NAME', 'ClientID', 'StrategyName']])

df = pd.concat(res)

sql = "SELECT A.USER_ID, A.USER_NAME, B.STRPURSTARTTIME \
FROM TRM2200 AS A LEFT OUTER JOIN TRM2300 B \
ON A.SEQ = B.TRM2200_SEQ \
LEFT OUTER JOIN TRM1300 C \
ON B.TRM1300_SEQ = C.STRSEQ \
LEFT OUTER JOIN TRM1310 D \
ON D.STRSEQ = C.STRSEQ \
LEFT OUTER JOIN BASECODE E \
ON E.TOTAL_CODE = D.STRRANGECODE \
LEFT OUTER JOIN BASECODE F \
ON F.TOTAL_CODE = B.STRPURCODE \
LEFT OUTER JOIN BASECODE G \
ON G.TOTAL_CODE = B.STRSELLCODE \
LEFT OUTER JOIN BASECODE H \
ON H.TOTAL_CODE = B.STRPURVAL \
LEFT OUTER JOIN BASECODE I \
ON I.TOTAL_CODE = B.STRSELLVAL \
WHERE STRNAME='Master.G';"

clients = maria.showData(sql).drop_duplicates(subset='USER_ID', keep='last')
time_list = list(clients['STRPURSTARTTIME'])
time_list = ['0'+t if len(t)==4 else t for t in time_list]
clients['STRPURSTARTTIME'] = time_list
clients = clients[clients['STRPURSTARTTIME'] < '15:30'][['USER_ID', 'USER_NAME']]
clients.columns = ['ClientID', 'ClientName']

df = pd.merge(df, clients, on='ClientID')[['ClientID', 'ClientName', 'StrategyName', 'NAME']]

nowtime = str(datetime.datetime.now()).replace(' ', '_').replace(':', '.') + '.csv'
df.to_csv(nowtime, encoding='utf-8-sig', index=False)


    
    








C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['VVIP1', 'VVIP151', 'SVIP6', 'SVIP622', 'G', 'G50', 'E', 'E19', 'A', 'A50', 'C', 'C79', 'S', 'S28', 'N', 'N75', 'T', 'T36', "S'", "S'13"]


C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
boyoo.columns[1:3]

Index(['VVIP151', 'SVIP6'], dtype='object')

In [55]:
import cjw_maria
import pandas as pd
import datetime
from pykrx import stock

host = "15.165.29.213"
user = "lt_user"
password = "De4IjOY32e7o" 
db = "leaderstrading"
maria = cjw_maria.MariaDB(host, user, password, db)

sql = "SELECT * FROM trading_history WHERE clientid='dnsdydqn4' and StrategyName='Master.C' ORDER BY Name, CompleteTime ASC;"
ordered_df = maria.showData(sql)

after_processed_df = ordered_df.drop_duplicates(subset=['Name'], keep='last')[['Name', 'ID', 'Quantity', 'AvgBuyPrice', 'Status', 'TradingType']]

after_processed_df = after_processed_df[after_processed_df['TradingType']=='매수']

std_date = '20210303'
df1 = stock.get_market_ohlcv_by_ticker(std_date)['종가']
df2 = stock.get_market_ohlcv_by_ticker(std_date, market='KOSDAQ')['종가']
df3 = stock.get_market_ohlcv_by_ticker(std_date, market='KONEX')['종가']
price_df = pd.concat([df1, df2, df3]).reset_index(drop=False).rename(columns={'티커': 'ID', '종가': 'NowPrice'})
df = pd.merge(after_processed_df, price_df, on=['ID'])
df['수익률'] = ((df['NowPrice'] - df['AvgBuyPrice'].astype('float')) / df['AvgBuyPrice'].astype('float')) * 100
df['매수금액'] = (df['Quantity'].astype('float') * df['AvgBuyPrice'].astype('float')).astype('int')
df['예수금'] = (df['Quantity'].astype('float') * df['NowPrice']).astype('int')
df[['ID', 'Name', 'AvgBuyPrice', 'Quantity', 'NowPrice', '수익률', '매수금액', '예수금']]


,ID,Name,AvgBuyPrice,Quantity,NowPrice,수익률,매수금액,예수금
0,148140,비디아이,9110.000000000,328,8900,-2.305159,2988080,2919200


In [56]:
df[['ID', 'Name', 'AvgBuyPrice', 'Quantity', 'NowPrice', '수익률', '매수금액', '예수금']].to_excel('보유종목C.xlsx', index=False, encoding='utf-8-sig')

In [43]:
price_df

,ID,NowPrice
0,095570,3970
1,006840,31400
2,027410,5760
3,282330,169500
4,138930,6320
...,...,...
2541,242350,782
2542,331660,18350
2543,107640,6000
2544,284420,15700


In [45]:
df

,Name,ID,Quantity,AvgBuyPrice,Status,TradingType,NowPrice
0,AP위성,211270,264,15064,매도체결완료,매도,19350
1,CMG제약,058820,727,5495,매도체결완료,매도,4480
2,CS,065770,1381,2895,매도체결완료,매도,2630
3,DB하이텍,000990,60,66400.000000000,체결완료,매수,56400
4,DSC인베스트먼트,241520,617,6469.011345219,체결완료,매수,4520
...,...,...,...,...,...,...,...
307,형지엘리트,093240,413,4805.000000000,매도체결완료,매도,3820
308,화신정공,126640,1028,1935.000000000,매도체결완료,매도,1735
309,후성,093370,216,9260,매도체결완료,매도,10200
310,흥구석유,024060,244,8100.000000000,매도체결완료,매도,7170


,Name,ID,Quantity,AvgBuyPrice,Status,TradingType,NowPrice,수익률
0,AP위성,211270,264,15064,매도체결완료,매도,19350,28.451938
1,CMG제약,058820,727,5495,매도체결완료,매도,4480,-18.471338
2,CS,065770,1381,2895,매도체결완료,매도,2630,-9.153713
3,DB하이텍,000990,60,66400.000000000,체결완료,매수,56400,-15.060241
4,DSC인베스트먼트,241520,617,6469.011345219,체결완료,매수,4520,-30.128427
...,...,...,...,...,...,...,...,...
307,형지엘리트,093240,413,4805.000000000,매도체결완료,매도,3820,-20.499480
308,화신정공,126640,1028,1935.000000000,매도체결완료,매도,1735,-10.335917
309,후성,093370,216,9260,매도체결완료,매도,10200,10.151188
310,흥구석유,024060,244,8100.000000000,매도체결완료,매도,7170,-11.481481


In [ ]:
type()